In [3]:
s0 = PermutationGroupElement('(1)(2)')
r0 = PermutationGroupElement('(1,2)')
R0 = RibbonGraph(s0,r0); R0

Ribbon graph of genus 0 and 1 boundary components

In [ ]:
# ============================================================
# VERIFICACIÓN DEL ENTORNO
# ============================================================
# Esta celda verifica que todos los módulos necesarios estén disponibles

print("Verificando entorno de SageMath...")
print(f"Versión de SageMath: {version()}")
print(f"Versión de Python: {sys.version}")

# Verificar imports críticos
try:
    from sage.geometry.ribbon_graph import RibbonGraph
    print("✓ RibbonGraph disponible")
except ImportError as e:
    print(f"✗ Error al importar RibbonGraph: {e}")

try:
    from sage.groups.perm_gps.permgroup_element import PermutationGroupElement
    print("✓ PermutationGroupElement disponible")
except ImportError as e:
    print(f"✗ Error al importar PermutationGroupElement: {e}")

try:
    from sage.plot.plot3d.shapes2 import polygon3d, line3d
    print("✓ Funciones 3D (polygon3d, line3d) disponibles")
except ImportError as e:
    print(f"✗ Error al importar funciones 3D: {e}")

try:
    # Verificar que sphere está disponible globalmente
    test_sphere = sphere(center=(0,0,0), size=0.1)
    print("✓ sphere() disponible globalmente")
except Exception as e:
    print(f"✗ Error con sphere(): {e}")

try:
    import numpy as np
    print(f"✓ NumPy disponible (versión {np.__version__})")
except ImportError as e:
    print(f"✗ Error al importar NumPy: {e}")

print("\n" + "="*60)
print("Entorno verificado correctamente. Listo para trabajar!")
print("="*60)

# 📘 Ribbon Graphs - Sistema de Visualización

Este notebook te permite trabajar con **ribbon graphs** (grafos de cinta) usando SageMath.

## ⚡ Inicio Rápido

**IMPORTANTE**: Ejecuta las celdas en orden, de arriba hacia abajo.

1. **Verificación del entorno** ⬇️ (celda siguiente)
2. **Definición de clases** (clase `RibbonGraphVisualizer`)
3. **Ejemplos y visualizaciones**

---

In [ ]:
import numpy as np
from sage.geometry.ribbon_graph import RibbonGraph
from sage.groups.perm_gps.permgroup_element import PermutationGroupElement
from sage.plot.plot3d.shapes2 import polygon3d, line3d

class RibbonGraphVisualizer:
    """
    Clase para visualización y análisis de Ribbon Graphs.
    Combina la potencia combinatoria de SageMath con visualizaciones 2D/3D.
    """
    
    def __init__(self, sigma, rho):
        """
        Inicializa un ribbon graph.
        
        Args:
            sigma: Permutación de vértices (ciclos alrededor de vértices)
            rho: Permutación de aristas (involución que empareja dardos)
        """
        if isinstance(sigma, str):
            sigma = PermutationGroupElement(sigma)
        if isinstance(rho, str):
            rho = PermutationGroupElement(rho)
            
        self.sigma = sigma
        self.rho = rho
        self.ribbon_graph = RibbonGraph(sigma, rho)
        self.graph = self.ribbon_graph.graph()
        
    def invariantes(self):
        """Calcula invariantes topológicos del ribbon graph."""
        return {
            'genus': self.ribbon_graph.genus(),
            'caras': self.ribbon_graph.number_of_faces(),
            'vertices': self.graph.num_verts(),
            'aristas': self.graph.num_edges(),
            'componentes_frontera': self.ribbon_graph.number_of_boundaries(),
            'euler_char': self.ribbon_graph.euler_characteristic()
        }
    
    def mostrar_invariantes(self):
        """Imprime los invariantes de forma legible."""
        inv = self.invariantes()
        print("="*50)
        print("INVARIANTES DEL RIBBON GRAPH")
        print("="*50)
        print(f"Género (g):                    {inv['genus']}")
        print(f"Vértices (V):                  {inv['vertices']}")
        print(f"Aristas (E):                   {inv['aristas']}")
        print(f"Caras (F):                     {inv['caras']}")
        print(f"Componentes de frontera (b):  {inv['componentes_frontera']}")
        print(f"Característica de Euler (χ):  {inv['euler_char']}")
        print("="*50)
        
    def visualizar_2d(self, layout='spring', vertex_size=300, edge_width=2, 
                     mostrar_dardos=True, mostrar_labels=True):
        """
        Visualización 2D del ribbon graph.
        
        Args:
            layout: Tipo de layout ('spring', 'circular', 'planar')
            vertex_size: Tamaño de los vértices
            edge_width: Grosor de las aristas
            mostrar_dardos: Si mostrar orientación de dardos
            mostrar_labels: Si mostrar etiquetas
        """
        G = self.graph
        
        # Calcular posiciones
        if layout == 'circular':
            pos = G.layout_circular()
        elif layout == 'planar':
            try:
                pos = G.layout_planar()
            except:
                print("El grafo no es planar, usando spring layout")
                pos = G.layout_spring()
        else:
            pos = G.layout_spring()
        
        # Crear visualización usando el método de plot de SageMath
        return G.plot(pos=pos, vertex_size=vertex_size, edge_thickness=edge_width,
                     vertex_labels=mostrar_labels)
    
    def visualizar_3d_cintas(self, layout='spring', iterations=500, 
                            ancho_cinta=0.15, opacity=0.7):
        """
        Visualización 3D que muestra las 'cintas' del ribbon graph.
        
        Args:
            layout: Tipo de layout
            iterations: Iteraciones para el layout spring
            ancho_cinta: Ancho de las cintas visualizadas
            opacity: Opacidad de las cintas
        """
        G = self.graph
        
        # Layout 3D
        pos = G.layout(layout=layout, dim=3, iterations=iterations, seed=42)
        
        # Inicializar escena vacía
        escena = None
        
        # Dibujar Vértices como esferas
        # sphere() está disponible globalmente en SageMath
        for v in G.vertices():
            esfera = sphere(center=pos[v], size=0.15, color='red')
            if escena is None:
                escena = esfera
            else:
                escena += esfera
            
        # Dibujar Cintas
        visited = set()
        
        for u, v, _ in G.edges():
            # Evitar duplicar aristas
            key = tuple(sorted([str(u), str(v)]))
            if key in visited:
                continue
            visited.add(key)
            
            p1 = np.array(pos[u])
            p2 = np.array(pos[v])
            vec = p2 - p1
            
            # Vector perpendicular para el ancho de la cinta
            mid = (p1 + p2) / 2
            if np.linalg.norm(mid) > 1e-5:
                up = mid / np.linalg.norm(mid)
            else:
                up = np.array([0, 0, 1])
            
            perp = np.cross(vec, up)
            if np.linalg.norm(perp) > 1e-5:
                perp = perp / np.linalg.norm(perp) * ancho_cinta
            else:
                perp = np.array([ancho_cinta, 0, 0])
            
            # Polígono de la cinta
            pts = [
                tuple(p1 + perp),
                tuple(p2 + perp),
                tuple(p2 - perp),
                tuple(p1 - perp)
            ]
            
            escena += polygon3d(pts, color='cyan', opacity=opacity)
            escena += line3d([tuple(p1), tuple(p2)], color='black', thickness=3)
        
        return escena
    
    def exportar_info(self):
        """Exporta información completa del ribbon graph."""
        inv = self.invariantes()
        info = {
            'invariantes': inv,
            'sigma': str(self.sigma),
            'rho': str(self.rho),
            'vertices': list(self.graph.vertices()),
            'aristas': list(self.graph.edges()),
        }
        return info

# Ejemplo: Crear un ribbon graph simple
print("Ejemplo 1: Ribbon Graph simple")
sigma1 = PermutationGroupElement('(1,2,3)(4,5,6)')
rho1 = PermutationGroupElement('(1,4)(2,5)(3,6)')

viz1 = RibbonGraphVisualizer(sigma1, rho1)
viz1.mostrar_invariantes()

# Visualización 2D
print("\nGenerando visualización 2D...")
plot_2d = viz1.visualizar_2d(layout='spring', mostrar_labels=True)
plot_2d.show(figsize=8)

---

## 📦 Definición de Clases y Primer Ejemplo

Ejecuta la celda siguiente para:
- Definir la clase `RibbonGraphVisualizer`
- Crear el primer ejemplo (`viz1`)
- Ver la visualización 2D

**⚠️ IMPORTANTE**: Esta celda debe ejecutarse antes de las siguientes.

In [2]:
# Visualización 3D con cintas
print("Generando visualización 3D con cintas...")
escena_3d = viz1.visualizar_3d_cintas(iterations=500, ancho_cinta=0.2, opacity=0.8)
escena_3d.show(viewer='threejs', frame=False, online=True)

Generando visualización 3D con cintas...


NameError: name 'viz1' is not defined

---

## 🎨 Visualización 3D del Primer Ejemplo

Esta celda usa `viz1` que se definió en la celda anterior.

**Si obtienes un error "viz1 is not defined"**, vuelve a ejecutar la celda anterior primero.

In [ ]:
# Ejemplo 2: Ribbon Graph más complejo (Tetraedro en superficie)
print("\n" + "="*60)
print("Ejemplo 2: Ribbon Graph más complejo")
print("="*60)

# Tetraedro tiene 4 vértices, 6 aristas, 12 dardos
sigma2 = PermutationGroupElement('(1,2,3)(4,5,6)(7,8,9)(10,11,12)')
rho2 = PermutationGroupElement('(1,4)(2,7)(3,10)(5,8)(6,11)(9,12)')

viz2 = RibbonGraphVisualizer(sigma2, rho2)
viz2.mostrar_invariantes()

# Visualización 2D con layout circular
plot_2d_circular = viz2.visualizar_2d(layout='circular', vertex_size=500)
plot_2d_circular.show(figsize=10)

---

## 📐 Ejemplo 2: Grafo más Complejo (Tetraedro)

Este ejemplo crea un nuevo ribbon graph `viz2` y lo visualiza.

In [ ]:
# Visualización 3D del ejemplo complejo
print("Generando visualización 3D del tetraedro...")
escena_3d_tetra = viz2.visualizar_3d_cintas(iterations=1000, ancho_cinta=0.15, opacity=0.7)
escena_3d_tetra.show(viewer='threejs', frame=False, online=True)

In [ ]:
# ========================================================================
# HERRAMIENTAS AVANZADAS PARA INVESTIGACIÓN
# ========================================================================

class RibbonGraphTools:
    """Herramientas adicionales para investigación en ribbon graphs."""
    
    @staticmethod
    def desde_grafo_con_rotacion(vertices, aristas, rotaciones):
        """
        Construye un ribbon graph desde un grafo con rotaciones en vértices.
        
        Args:
            vertices: Lista de vértices
            aristas: Lista de tuplas (u, v) representando aristas
            rotaciones: Dict {vertice: [lista_ordenada_de_vecinos]}
        
        Returns:
            Tupla (sigma, rho) para construir el ribbon graph
        """
        # Crear mapeo de aristas a dardos
        dart_counter = 1
        dart_map = {}
        arista_to_darts = {}
        
        for i, (u, v) in enumerate(aristas):
            d1 = dart_counter
            d2 = dart_counter + 1
            dart_counter += 2
            
            arista_to_darts[i] = (d1, d2)
            dart_map[d1] = (u, v, 'out')
            dart_map[d2] = (v, u, 'out')
        
        # Construir sigma (permutación de vértices)
        sigma_cycles = []
        for v in vertices:
            if v not in rotaciones:
                continue
            
            ciclo = []
            for vecino in rotaciones[v]:
                # Encontrar el dardo que va de v a vecino
                for i, (u, w) in enumerate(aristas):
                    d1, d2 = arista_to_darts[i]
                    if u == v and w == vecino:
                        ciclo.append(d1)
                        break
                    elif w == v and u == vecino:
                        ciclo.append(d2)
                        break
            
            if len(ciclo) > 0:
                sigma_cycles.append(ciclo)
        
        # Construir rho (permutación de aristas)
        rho_pairs = []
        for d1, d2 in arista_to_darts.values():
            rho_pairs.append((d1, d2))
        
        return sigma_cycles, rho_pairs
    
    @staticmethod
    def comparar_grafos(viz1, viz2):
        """Compara dos ribbon graphs y muestra sus diferencias."""
        inv1 = viz1.invariantes()
        inv2 = viz2.invariantes()
        
        print("="*70)
        print("COMPARACIÓN DE RIBBON GRAPHS")
        print("="*70)
        print(f"{'Invariante':<25} {'Grafo 1':>15} {'Grafo 2':>15} {'Diferencia':>15}")
        print("-"*70)
        
        for key in inv1.keys():
            diff = inv2[key] - inv1[key] if isinstance(inv1[key], (int, float)) else "N/A"
            print(f"{key:<25} {str(inv1[key]):>15} {str(inv2[key]):>15} {str(diff):>15}")
        print("="*70)
    
    @staticmethod
    def construir_bouquet(n):
        """
        Construye un bouquet de n loops (n aristas conectadas a un solo vértice).
        
        Args:
            n: Número de loops
            
        Returns:
            RibbonGraphVisualizer del bouquet
        """
        # Un vértice con n loops
        # Dardos: 1, 2, ..., 2n
        ciclo_vertice = list(range(1, 2*n + 1))
        sigma_str = '(' + ','.join(map(str, ciclo_vertice)) + ')'
        
        # Emparejar dardos: (1,2), (3,4), ..., (2n-1, 2n)
        rho_pairs = ['(' + str(2*i-1) + ',' + str(2*i) + ')' for i in range(1, n+1)]
        rho_str = ''.join(rho_pairs)
        
        sigma = PermutationGroupElement(sigma_str)
        rho = PermutationGroupElement(rho_str)
        
        return RibbonGraphVisualizer(sigma, rho)
    
    @staticmethod
    def construir_ciclo(n):
        """
        Construye un ciclo de longitud n.
        
        Args:
            n: Longitud del ciclo
            
        Returns:
            RibbonGraphVisualizer del ciclo
        """
        # n vértices, cada uno con grado 2
        # Dardos: 1, 2, ..., 2n
        
        # Sigma: cada vértice tiene dos dardos consecutivos
        sigma_cycles = []
        for i in range(n):
            d1 = 2*i + 1
            d2 = 2*i + 2
            sigma_cycles.append(f"({d1},{d2})")
        sigma_str = ''.join(sigma_cycles)
        
        # Rho: conectar dardos en círculo
        rho_pairs = []
        for i in range(n):
            d_out = 2*i + 2
            d_in = (2*i + 3) if i < n-1 else 1
            rho_pairs.append(f"({d_out},{d_in})")
        rho_str = ''.join(rho_pairs)
        
        sigma = PermutationGroupElement(sigma_str)
        rho = PermutationGroupElement(rho_str)
        
        return RibbonGraphVisualizer(sigma, rho)

# Ejemplo: Bouquet de 3 loops
print("\n" + "="*60)
print("Ejemplo 3: Bouquet de 3 loops")
print("="*60)

viz_bouquet = RibbonGraphTools.construir_bouquet(3)
viz_bouquet.mostrar_invariantes()

plot_bouquet = viz_bouquet.visualizar_2d(layout='spring', vertex_size=800)
plot_bouquet.show(figsize=8)

---

## 🔧 Herramientas Avanzadas

Esta sección define `RibbonGraphTools` con métodos para:
- Construir grafos comunes (bouquets, ciclos)
- Comparar diferentes ribbon graphs
- Construir desde representaciones alternativas

In [ ]:
# Ejemplo: Ciclo de longitud 5
print("\n" + "="*60)
print("Ejemplo 4: Ciclo de longitud 5")
print("="*60)

viz_ciclo = RibbonGraphTools.construir_ciclo(5)
viz_ciclo.mostrar_invariantes()

plot_ciclo = viz_ciclo.visualizar_2d(layout='circular', vertex_size=600)
plot_ciclo.show(figsize=8)

In [ ]:
# Comparación entre dos ribbon graphs
print("\n" + "="*60)
print("COMPARACIÓN: Bouquet vs Ciclo")
print("="*60)

RibbonGraphTools.comparar_grafos(viz_bouquet, viz_ciclo)

# Sistema de Visualización de Ribbon Graphs

## 🎯 Características

Este notebook implementa un sistema completo para trabajar con **ribbon graphs** (grafos de cinta) que combina:

- ✅ **Combinatoria robusta** usando SageMath
- ✅ **Visualización 2D y 3D** con múltiples layouts
- ✅ **Cálculo de invariantes topológicos** (género, caras, componentes de frontera, etc.)
- ✅ **Construcciones comunes** (bouquets, ciclos, etc.)
- ✅ **Herramientas de comparación** entre grafos

---

## 📚 Uso Básico

### 1. Crear un Ribbon Graph desde Permutaciones

```python
sigma = PermutationGroupElement('(1,2,3)(4,5,6)')
rho = PermutationGroupElement('(1,4)(2,5)(3,6)')
viz = RibbonGraphVisualizer(sigma, rho)
```

### 2. Calcular Invariantes

```python
viz.mostrar_invariantes()
inv = viz.invariantes()  # Retorna un diccionario
```

### 3. Visualización 2D

```python
plot = viz.visualizar_2d(layout='spring', vertex_size=500)
plot.show(figsize=10)
```

Layouts disponibles: `'spring'`, `'circular'`, `'planar'`

### 4. Visualización 3D con Cintas

```python
escena = viz.visualizar_3d_cintas(ancho_cinta=0.2, opacity=0.7)
escena.show(viewer='threejs', online=True)
```

---

## 🔬 Herramientas para Investigación

### Construcciones Comunes

```python
# Bouquet de n loops
viz_bouquet = RibbonGraphTools.construir_bouquet(n=5)

# Ciclo de longitud n
viz_ciclo = RibbonGraphTools.construir_ciclo(n=7)
```

### Comparación de Grafos

```python
RibbonGraphTools.comparar_grafos(viz1, viz2)
```

### Exportar Información

```python
info = viz.exportar_info()
# Retorna un diccionario con toda la información del ribbon graph
```

---

## 🚀 Extensiones Futuras

Para investigación avanzada, considera agregar:

1. **Operaciones en ribbon graphs**:
   - Contracción de aristas
   - Dualidad
   - Truncamiento de vértices

2. **Invariantes adicionales**:
   - Polinomios (Tutte, Jones, etc.)
   - Homología
   - Grupos fundamentales

3. **Visualización interactiva**:
   - Integración con Plotly para interactividad
   - Animaciones de operaciones
   - Exportar a formatos web (D3.js, Three.js)

4. **Interfaz con otras herramientas**:
   - Exportar a NetworkX para análisis adicional
   - Conexión con SnapPy para superficies hiperbólicas
   - Exportar a formatos de geometría computacional (OFF, STL)

---

## 📖 Referencias

- **Ribbon Graphs en SageMath**: [Documentación oficial](https://doc.sagemath.org/html/en/reference/geometry/sage/geometry/ribbon_graph.html)
- **Teoría**: "Ribbon graphs and their invariants derived from quantum groups" - Reshetikhin & Turaev (1990)

---

## 💡 Tips para la Clase

- Usa `layout='circular'` para grafos simétricos
- `layout='spring'` funciona mejor para grafos densos
- Aumenta `iterations` en 3D para layouts más estables
- Usa `opacity=0.5` para ver el interior de estructuras complejas